## Raw Project

### Train Chatbot

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
# words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    # pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
# model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(200, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


41 documents
6 classes ['daftar', 'identitas', 'poli', 'terimakasih', 'ucapan', 'waktu']
48 unique lemmatized words ['anak', 'assalamualaikum', 'bedah', 'daftar', 'dalam', 'hai', 'hari', 'hay', 'helo', 'hey', 'ingin', 'jalan', 'jumat', 'kamis', 'kasih', 'ke', 'makasih', 'malam', 'matur', 'mau', 'minggu', 'nama', 'no', 'norm', 'nuwun', 'pagi', 'pasien', 'pol', 'rabu', 'rawat', 'registrasi', 'rm', 'sabtu', 'sakit', 'salam', 'saraf', 'saya', 'selamat', 'selasa', 'senin', 'siang', 'sore', 'suwun', 'syaraf', 'tanggal', 'terima', 'terimakasih', 'tuju']
Training data created
Epoch 1/200
41/41 [==============================] - 0s 3ms/step - loss: 1.7606 - accuracy: 0.1463
Epoch 2/200
41/41 [==============================] - 0s 545us/step - loss: 1.7486 - accuracy: 0.3171
Epoch 3/200
41/41 [==============================] - 0s 622us/step - loss: 1.6518 - accuracy: 0.4146
Epoch 4/200
41/41 [==============================] - 0s 666us/step - loss: 1.6510 - accuracy: 0.3659
Epoch 5/200
41/41 [====

41/41 [==============================] - 0s 690us/step - loss: 0.0180 - accuracy: 1.0000
Epoch 76/200
41/41 [==============================] - 0s 739us/step - loss: 0.0223 - accuracy: 1.0000
Epoch 77/200
41/41 [==============================] - 0s 753us/step - loss: 0.0363 - accuracy: 0.9756
Epoch 78/200
41/41 [==============================] - 0s 831us/step - loss: 0.0218 - accuracy: 1.0000
Epoch 79/200
41/41 [==============================] - 0s 880us/step - loss: 0.0226 - accuracy: 1.0000
Epoch 80/200
41/41 [==============================] - 0s 802us/step - loss: 0.0091 - accuracy: 1.0000
Epoch 81/200
41/41 [==============================] - 0s 747us/step - loss: 0.0141 - accuracy: 1.0000
Epoch 82/200
41/41 [==============================] - 0s 710us/step - loss: 0.0080 - accuracy: 1.0000
Epoch 83/200
41/41 [==============================] - 0s 748us/step - loss: 0.0122 - accuracy: 1.0000
Epoch 84/200
41/41 [==============================] - 0s 737us/step - loss: 0.0116 - accuracy: 

41/41 [==============================] - 0s 780us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 155/200
41/41 [==============================] - 0s 920us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 156/200
41/41 [==============================] - 0s 895us/step - loss: 0.0123 - accuracy: 1.0000
Epoch 157/200
41/41 [==============================] - 0s 777us/step - loss: 0.0064 - accuracy: 1.0000
Epoch 158/200
41/41 [==============================] - 0s 749us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 159/200
41/41 [==============================] - 0s 825us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 160/200
41/41 [==============================] - 0s 822us/step - loss: 0.0268 - accuracy: 0.9756
Epoch 161/200
41/41 [==============================] - 0s 825us/step - loss: 0.0084 - accuracy: 1.0000
Epoch 162/200
41/41 [==============================] - 0s 849us/step - loss: 0.0112 - accuracy: 1.0000
Epoch 163/200
41/41 [==============================] - 0s 852us/step - loss: 0.0056 - a

### Process new sentence

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    # sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

### Testing

In [15]:
print(chatbot_response('Matur nuwun'))

Iya, kembali kasih


## Trial MongoDB

### Example 

In [28]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["pythonmongo"]
mycol = mydb["customers"]

# insert one object

mydict = {"name": "Adam", "address":"Laweyan"}
x1 = mycol.insert_one(mydict)

print(x1)
print(x1.inserted_id)

# insert multiple documents into a collection

mylist = [
  { "name": "Amy", "address": "Apple st 652"},
  { "name": "Hannah", "address": "Mountain 21"},
  { "name": "Michael", "address": "Valley 345"},
  { "name": "Sandy", "address": "Ocean blvd 2"},
  { "name": "Betty", "address": "Green Grass 1"},
  { "name": "Richard", "address": "Sky st 331"},
  { "name": "Susan", "address": "One way 98"},
  { "name": "Vicky", "address": "Yellow Garden 2"},
  { "name": "Ben", "address": "Park Lane 38"},
  { "name": "William", "address": "Central st 954"},
  { "name": "Chuck", "address": "Main Road 989"},
  { "name": "Viola", "address": "Sideway 1633"}
]

x2 = mycol.insert_many(mylist)

print(x2)
print(x2.inserted_ids)

# Find One

x3 = mycol.find_one()
print(x3)

# Find All

for x4 in mycol.find():
    print(x4)

5ea3e1c628bbb7957263404d
[ObjectId('5ea3e1c628bbb7957263404e'), ObjectId('5ea3e1c628bbb7957263404f'), ObjectId('5ea3e1c628bbb79572634050'), ObjectId('5ea3e1c628bbb79572634051'), ObjectId('5ea3e1c628bbb79572634052'), ObjectId('5ea3e1c628bbb79572634053'), ObjectId('5ea3e1c628bbb79572634054'), ObjectId('5ea3e1c628bbb79572634055'), ObjectId('5ea3e1c628bbb79572634056'), ObjectId('5ea3e1c628bbb79572634057'), ObjectId('5ea3e1c628bbb79572634058'), ObjectId('5ea3e1c628bbb79572634059')]
{'_id': ObjectId('5ea3e1c628bbb7957263404d'), 'name': 'Adam', 'address': 'Laweyan'}
{'_id': ObjectId('5ea3e1c628bbb7957263404d'), 'name': 'Adam', 'address': 'Laweyan'}
{'_id': ObjectId('5ea3e1c628bbb7957263404e'), 'name': 'Amy', 'address': 'Apple st 652'}
{'_id': ObjectId('5ea3e1c628bbb7957263404f'), 'name': 'Hannah', 'address': 'Mountain 21'}
{'_id': ObjectId('5ea3e1c628bbb79572634050'), 'name': 'Michael', 'address': 'Valley 345'}
{'_id': ObjectId('5ea3e1c628bbb79572634051'), 'name': 'Sandy', 'address': 'Ocean b

### Connect Intents to MongoDB

In [87]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["pythonmongo"]
mycol = mydb["intents"]

myIntents = [
    {"tag": "ucapan",
     "patterns": ["Assalamualaikum", "Selamat pagi", "Selamat siang","Selamat sore","Selamat malam", "Salam", "Hai", "Helo", "Hey", "Hay"],
     "responses": ["Salam, ada yang bisa kami bantu?", "Salam, aku Bening"],
     "context": [""]
    },
    {"tag": "terimakasih",
     "patterns": ["Matur nuwun", "Terima kasih", "Makasih","terimakasih","Suwun"],
     "responses": ["Baik, sama-sama", "Iya, kembali kasih", "Nggih, sami-sami"],
     "context": [""]
    },
    {"tag": "daftar",
     "patterns": ["Saya ingin mendaftar", "Daftar", "Saya ingin registrasi", "Daftar Poli Rawat Jalan"],
     "responses": ["Baik, silahkan masukkan Nama dan Nomor RM"],
     "context": [""]
    },
    {"tag": "identitas",
     "patterns": ["Nama saya", "Nama Pasien", "Pasien", "No RM", "norm"],
     "responses": ["Baik, tujuan ke Poli mana?"],
     "context": [""]
    },
    {"tag": "poli",
     "patterns": ["Saya ingin mendaftar ke Poli", "Tujuan ke Poli", "Mau ke Poli", "Poli Bedah", "Poli Penyakit Dalam", "Poli Anak", "Poli Syaraf", "Poli Saraf"],
     "responses": ["Baik, kapan Anda akan berkunjung?"],
     "context": [""]
    },
    {"tag": "waktu",
     "patterns": ["Hari", "Senin", "Tanggal", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu", "Minggu"],
     "responses": ["Oke, dokternya bisa kok hari itu"],
     "context": [""]
    }
]

z1 = mycol.insert_many(myIntents)

print(z1)
print(z1.inserted_ids)

[ObjectId('5ea3ef9b28bbb79572634062'), ObjectId('5ea3ef9b28bbb79572634063'), ObjectId('5ea3ef9b28bbb79572634064'), ObjectId('5ea3ef9b28bbb79572634065'), ObjectId('5ea3ef9b28bbb79572634066'), ObjectId('5ea3ef9b28bbb79572634067')]


#### Get All Intents from MongoDB 

In [81]:
import numpy
dictIntents = {}
intentsData = []

for z2 in mycol.find():
    intentsData.append(z2)
    
dictIntents["intents"] = intentsData
print(dictIntents)
# print(dictIntents["intents"]["_id"])

{'intents': [{'_id': ObjectId('5ea3e26f28bbb7957263405b'), 'tag': 'ucapan', 'patterns': ['Assalamualaikum', 'Selamat pagi', 'Selamat siang', 'Selamat sore', 'Selamat malam', 'Salam', 'Hai', 'Helo', 'Hey', 'Hay'], 'responses': ['Salam, ada yang bisa kami bantu?', 'Salam, aku Bening'], 'context': ['']}, {'_id': ObjectId('5ea3e26f28bbb7957263405c'), 'tag': 'terimakasih', 'patterns': ['Matur nuwun', 'Terima kasih', 'Makasih', 'terimakasih', 'Suwun'], 'responses': ['Baik, sama-sama', 'Iya, kembali kasih', 'Nggih, sami-sami'], 'context': ['']}, {'_id': ObjectId('5ea3e26f28bbb7957263405d'), 'tag': 'daftar', 'patterns': ['Saya ingin mendaftar', 'Daftar', 'Saya ingin registrasi', 'Daftar Poli Rawat Jalan'], 'responses': ['Baik, silahkan masukkan Nama dan Nomor RM'], 'context': ['']}, {'_id': ObjectId('5ea3e26f28bbb7957263405e'), 'tag': 'identitas', 'patterns': ['Nama saya', 'Nama Pasien', 'Pasien', 'No RM', 'norm'], 'responses': ['Baik, tujuan ke Poli mana?'], 'context': ['']}, {'_id': ObjectId

#### Train Chatbot

In [88]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import json
import pickle

from bson import ObjectId

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# JSON Encoder using BSON
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = dictIntents

intents = json.dumps(data_file, cls=JSONEncoder)
intents = json.loads(intents)
print(intents)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
# words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    # pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
# model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(200, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

{'intents': [{'_id': '5ea3e26f28bbb7957263405b', 'tag': 'ucapan', 'patterns': ['Assalamualaikum', 'Selamat pagi', 'Selamat siang', 'Selamat sore', 'Selamat malam', 'Salam', 'Hai', 'Helo', 'Hey', 'Hay'], 'responses': ['Salam, ada yang bisa kami bantu?', 'Salam, aku Bening'], 'context': ['']}, {'_id': '5ea3e26f28bbb7957263405c', 'tag': 'terimakasih', 'patterns': ['Matur nuwun', 'Terima kasih', 'Makasih', 'terimakasih', 'Suwun'], 'responses': ['Baik, sama-sama', 'Iya, kembali kasih', 'Nggih, sami-sami'], 'context': ['']}, {'_id': '5ea3e26f28bbb7957263405d', 'tag': 'daftar', 'patterns': ['Saya ingin mendaftar', 'Daftar', 'Saya ingin registrasi', 'Daftar Poli Rawat Jalan'], 'responses': ['Baik, silahkan masukkan Nama dan Nomor RM'], 'context': ['']}, {'_id': '5ea3e26f28bbb7957263405e', 'tag': 'identitas', 'patterns': ['Nama saya', 'Nama Pasien', 'Pasien', 'No RM', 'norm'], 'responses': ['Baik, tujuan ke Poli mana?'], 'context': ['']}, {'_id': '5ea3e26f28bbb7957263405f', 'tag': 'poli', 'patt

41/41 [==============================] - 0s 1ms/step - loss: 0.0311 - accuracy: 1.0000
Epoch 63/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 64/200
41/41 [==============================] - 0s 969us/step - loss: 0.0309 - accuracy: 1.0000
Epoch 65/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 66/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 67/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 68/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0132 - accuracy: 1.0000
Epoch 69/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0449 - accuracy: 1.0000
Epoch 70/200
41/41 [==============================] - 0s 936us/step - loss: 0.0161 - accuracy: 1.0000
Epoch 71/200
41/41 [==============================] - 0s 866us/step - loss: 0.0140 - accuracy: 1.0000
Epoch 7

41/41 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 143/200
41/41 [==============================] - 0s 994us/step - loss: 0.0366 - accuracy: 0.9756
Epoch 144/200
41/41 [==============================] - 0s 903us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 145/200
41/41 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 146/200
41/41 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 147/200
41/41 [==============================] - 0s 929us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 148/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 149/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 150/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 151/200
41/41 [==============================] - 0s 1ms/step - loss: 0.0070 - accuracy: 1.000

#### Execusion 

In [89]:
import nltk
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
# intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    # sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

#### Testing

In [92]:
print(chatbot_response('Saya ingin mendaftar poli Anak'))

Baik, kapan Anda akan berkunjung?
